## Environment Setup

In [1]:
import os
from dotenv import load_dotenv

# Set environment variables in the .env file.
load_dotenv()

COMPLETION_MODEL = os.environ["OPENAI_COMPLETION_MODEL"]
COMPLETION_DEPLOYMENT = os.environ["OPENAI_COMPLETION_DEPLOYMENT"]
CHAT_MODEL = os.environ["OPENAI_CHAT_MODEL"]
CHAT_DEPLOYMENT = os.environ["OPENAI_CHAT_DEPLOYMENT"]
OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]

In [2]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationChain

llm = AzureOpenAI(
    model_name=COMPLETION_MODEL,
    deployment_name=COMPLETION_DEPLOYMENT,
    #temperature=0.3,
    verbose=True
)

chat = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION,
    #temperature=0.3,
    verbose=True
)

In [19]:
prompt_template = """You are an AI assistant that helps the person answer a question based on the data below.
Answer the question below using the data from these records.
Show your work step by step.

# Data
{data}

# Conversation
Question: {question}
Answer:
"""

In [24]:
data = """Name,Gender,Age
John,M,13
Steve,M,25
Sarah,F,18
Jess,F,45
Will,M,50
"""

In [39]:
from langchain.tools import tool  

@tool
def compareValues(comparisonQuestion: str) -> bool:
    '''
        input string comparisonQuestion is a question in natural language about comparing two numerical values.
        For example, a question could be:
        "Is 5 greater than 3?"
    '''

    result = llm("Respond with 1 or 0. " + comparisonQuestion)

    return result == 1

In [40]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain

#from langchain.agents import load_tools
#tools = load_tools(["llm-math"], llm=llm)


llm_math_chain = LLMMathChain.from_llm(llm=chat, verbose=True)
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="Useful for when you need to answer questions using math."
    ),
    Tool(
        name="CompareValues",
        func=compareValues,
        description="Useful to comparing two numerical values."
    )
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

prompt = prompt_template.format(
    data=data,
    question="Who are older than 30 years old?"
)

agent.run(prompt)



> Entering new AgentExecutor chain...
 I need to compare the ages to 30 and see which are larger.
Action: CompareValues
Action Input: 30 and the Ages of the people in the records
Observation: False
Thought: I need to look through each record to see which records have an age larger than 30.
Action: CompareValues
Action Input: 30 and John's Age (13)
Observation: False
Thought: I need to look through each record to see which records have an age larger than 30.
Action: CompareValues
Action Input: 30 and Steve's Age (25)
Observation: False
Thought: I need to look through each record to see which records have an age larger than 30.
Action: CompareValues
Action Input: 30 and Sarah's Age (18)
Observation: False
Thought: I need to look through each record to see which records have an age larger than 30.
Action: CompareValues
Action Input: 30 and Jess's Age (45)
Observation: False
Thought: I need to look through each record to see which records have an age larger than 30.
Action: CompareValues

'Steve, Sarah, Jess, and Will are older than 30 years old.'